![header](images/header.jpg)

# Predicting Condition of Tanzanian Water Wells

**Author:** [Jonathan Fetterolf](mailto:jonathan.fetterolf@gmail.com)

## Overview

## Business Understanding and Business Problem
Using data from each waterpoint, can you predict which pumps are functional, which need some repairs, and which don't work at all? Data driven predictions will lead to a better maintenance operations and will ensure clean and potable water is available to communities across Tanzania. 

## Data Understanding
Data for this project is from [Taarifa](http://taarifa.org/) and the [Tanzanian Ministry of Water](http://maji.go.tz/).

## Exploratory Data Analyis

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

from sklearn.dummy import DummyClassifier

## Import Initial Data

In [2]:
df = pd.read_csv('data/training_set_values.csv', index_col='id')

## Functions


Creating a function and transformer that returns numeric columns. 

In [3]:
def grab_numeric(df):
    return df.select_dtypes(include=['float', 'int'])

In [4]:
GrabNumeric = FunctionTransformer(grab_numeric)

Numeric Pipeline

In [5]:
numeric_pipe = Pipeline([
    ('numeric_impute', SimpleImputer(strategy='mean')),
    ('ss', StandardScaler())
])

In [6]:
def null_info(col_name, df_name=df):
    '''
    give this function a column name and name of the DataFrame to
    receive a printout with how many null values the column contains,
    the percentage of missing values in that column, the top 5 values
    of the column, and how many unique values are in the column. 
    '''
    percent = round((df_name[col_name].isna().sum() / df_length) * 100, ndigits=2)
    values = df_name[col_name].value_counts().head()
    num_unique = len(list(df_name[col_name].unique()))
    col_type = df_name[col_name].dtypes
    total_nulls = df_name[col_name].isna().sum()
    return print(f'''Column Name: '{col_name}'\n
This column consists of {col_type}s and has {total_nulls} null values.\n
The percentage of missing values in the {col_name} \
column is {percent}%.\n
The top values in {col_name} are:\n
{values}\n
There are {num_unique} unique values in this column.
    ''')

## Data

In [7]:
df_length = len(df)
df_length

59400

In [8]:
col_names = list(df.columns)
num_cols = len(col_names)

print(f'The column names are: {col_names}\n Number of columns: {num_cols}')

The column names are: ['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']
 Number of columns: 39


### About the columns:
amount_tsh - Total static head (amount water available to waterpoint)<br>
date_recorded - The date the row was entered<br>
funder - Who funded the well<br>
gps_height - Altitude of the well<br>
installer - Organization that installed the well<br>
longitude - GPS coordinate<br>
latitude - GPS coordinate<br>
wpt_name - Name of the waterpoint if there is one<br>
num_private - <br>
basin - Geographic water basin<br>
subvillage - Geographic location<br>
region - Geographic location<br>
region_code - Geographic location (coded)<br>
district_code - Geographic location (coded)<br>
lga - Geographic location<br>
ward - Geographic location<br>
population - Population around the well<br>
public_meeting - True/False<br>
recorded_by - Group entering this row of data<br>
scheme_management - Who operates the waterpoint<br>
scheme_name - Who operates the waterpoint<br>
permit - If the waterpoint is permitted<br>
construction_year - Year the waterpoint was constructed<br>
extraction_type - The kind of extraction the waterpoint uses<br>
extraction_type_group - The kind of extraction the waterpoint uses<br>
extraction_type_class - The kind of extraction the waterpoint uses<br>
management - How the waterpoint is managed<br>
management_group - How the waterpoint is managed<br>
payment - What the water costs<br>
payment_type - What the water costs<br>
water_quality - The quality of the water<br>
quality_group - The quality of the water<br>
quantity - The quantity of water<br>
quantity_group - The quantity of water<br>
source - The source of the water<br>
source_type - The source of the water<br>
source_class - The source of the water<br>
waterpoint_type - The kind of waterpoint<br>
waterpoint_type_group - The kind of waterpoint<br>

In [9]:
# let's see what we're working with
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


The condition labels are in a separate DataFrame so we need to bring them in.

In [10]:
df_target = pd.read_csv('data/training_set_labels.csv', index_col='id')

In [11]:
df_target.columns

Index(['status_group'], dtype='object')

In [12]:
df_target.info(),
df_target.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 928.1+ KB


,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [13]:
# looks like we have the same number of rows.
df_target.shape, df.shape

((59400, 1), (59400, 39))

In [14]:
df = df.join(df_target)

In [15]:
# okay, our data is consolidated
df.shape

(59400, 40)

In [16]:
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


What are we looking at?

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

In [18]:
df.index.value_counts().sum()

59400

The index column, 'id' is a unique identifier for each row in the DataFrame

In [19]:
df.wpt_name.value_counts()

none                           3563
Shuleni                        1748
Zahanati                        830
Msikitini                       535
Kanisani                        323
                               ... 
Kwa Oldiani Gara                  1
Kwa Mama Shedi                    1
Kwamtemele                        1
Shule Ya Msingi Kinang'Weli       1
Kwa Mzee Jaba                     1
Name: wpt_name, Length: 37400, dtype: int64

The 'wpt_name' column is interesting because most waterpoints have names and some of those names are repeated and show up frequently.

In [20]:
df.num_private.value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

In [21]:
percent_of_zeroes = round(100 * (df.num_private.value_counts()[0]/len(df.num_private)))
print(f'Zero is the most common value in "num_private" and it makes up approximately {percent_of_zeroes}% of entries.')

Zero is the most common value in "num_private" and it makes up approximately 99.0% of entries.


All of the following columns describe the geographic location: 
'longitude', 'latitude', 'basin', 'subvillage', 'region', 'region_code', 'distric_code', 'lga', 'ward'

In [22]:
df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


I need to look into the 'construction_year' column because it has a minimum year of '0'.

In [23]:
year_zero_percent = round(100 * df.construction_year.value_counts(ascending=False)\
      [0]/len(df.construction_year), ndigits=2)
print(f'{year_zero_percent}% of the values in the column are zero')

34.86% of the values in the column are zero


I will have to remove all values of zero, filling them as null values and decide how to proceed when dealing with other null values.

In [24]:
df['construction_year'] = df['construction_year'].replace({0:pd.NA})

In [25]:
df.construction_year.head()

id
69572    1999
8776     2010
34310    2009
67743    1986
19728    <NA>
Name: construction_year, dtype: object

The 'date_recorded' and 'construction_year' columns need to be converted to datetime pandas.

### Addressing Null Values

In [26]:
df.isna().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year        20709
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

#### 'funder'

In [27]:
null_info('funder')

Column Name: 'funder'

This column consists of objects and has 3635 null values.

The percentage of missing values in the funder column is 6.12%.

The top values in funder are:

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Name: funder, dtype: int64

There are 1898 unique values in this column.
    


#### 'installer'

In [28]:
null_info('installer')

Column Name: 'installer'

This column consists of objects and has 3655 null values.

The percentage of missing values in the installer column is 6.15%.

The top values in installer are:

DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
Name: installer, dtype: int64

There are 2146 unique values in this column.
    


#### 'subvillage'

In [29]:
null_info('subvillage')

Column Name: 'subvillage'

This column consists of objects and has 371 null values.

The percentage of missing values in the subvillage column is 0.62%.

The top values in subvillage are:

Madukani    508
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
Name: subvillage, dtype: int64

There are 19288 unique values in this column.
    


#### 'public_meeting'

In [30]:
null_info('public_meeting')

Column Name: 'public_meeting'

This column consists of objects and has 3334 null values.

The percentage of missing values in the public_meeting column is 5.61%.

The top values in public_meeting are:

True     51011
False     5055
Name: public_meeting, dtype: int64

There are 3 unique values in this column.
    


#### 'scheme_management'

In [31]:
null_info('scheme_management')

Column Name: 'scheme_management'

This column consists of objects and has 3877 null values.

The percentage of missing values in the scheme_management column is 6.53%.

The top values in scheme_management are:

VWC                36793
WUG                 5206
Water authority     3153
WUA                 2883
Water Board         2748
Name: scheme_management, dtype: int64

There are 13 unique values in this column.
    


#### 'scheme_name'

In [32]:
null_info('scheme_name')

Column Name: 'scheme_name'

This column consists of objects and has 28166 null values.

The percentage of missing values in the scheme_name column is 47.42%.

The top values in scheme_name are:

K                682
None             644
Borehole         546
Chalinze wate    405
M                400
Name: scheme_name, dtype: int64

There are 2697 unique values in this column.
    


I will drop this column because over 47% of the rows have missing data and the 'scheme_managment' column has the same description of, "Who operates the waterpoint."

#### 'construction_year'

In [33]:
null_info('construction_year', df)

Column Name: 'construction_year'

This column consists of objects and has 20709 null values.

The percentage of missing values in the construction_year column is 34.86%.

The top values in construction_year are:

2010    2645
2008    2613
2009    2533
2000    2091
2007    1587
Name: construction_year, dtype: int64

There are 55 unique values in this column.
    


#### 'permit'

In [35]:
null_info('permit', df)

Column Name: 'permit'

This column consists of objects and has 3056 null values.

The percentage of missing values in the permit column is 5.14%.

The top values in permit are:

True     38852
False    17492
Name: permit, dtype: int64

There are 3 unique values in this column.
    


## Baseline Model
1st Model - "Dummy" Model

In [36]:
X = df.drop(['scheme_name', 'status_group'], axis=1)
y = df.status_group

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [37]:
X.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


We will predict the most frequently occuring value in 'status_group' for each row. 

In [38]:
y.value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [39]:
dummy_model = DummyClassifier(strategy='most_frequent')

In [47]:
dummy_model.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [48]:
dummy_predictions = dummy_model.predict(X_train)

In [49]:
cv_results = cross_val_score(dummy_model, X_train, y_train, cv=5)
cv_results

array([0.54231201, 0.54242424, 0.54231201, 0.54231201, 0.54231201])

In [50]:
np.mean(cv_results)

0.5423344556677889

In [51]:
dummy_model.score(X_train, y_train)

0.542334455667789

## First Simple Model

In [40]:
X = df.drop('scheme_name', axis=1)